# 2. Metadata Analysis
## 2.1 Analysis over the countries and regions

Let's first prepare our packages and data for analysis. Imported libraries such as `matplotlib`, `seaborn` and `plotly` will be used for data visualization.

In [38]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

path = '/Users/delete4ever/Desktop/Digital Literary Analysis/codes/' 
all_files = glob.glob(f"/Users/delete4ever/Desktop/Digital Literary Analysis/codes/billboard_hot_100_*.csv")

# Read each file and append it to a list
li = []
for filename in all_files:
    df = pd.read_csv(filename, usecols=['Nationality or Origin'], index_col=None)  # Adjust the usecols parameter if the nationality column is not the 4th
    year = int(filename[-8:-4])  # Extract the year from the filename
    df['year'] = year  # Add a 'year' column based on the filename
    li.append(df)

# Combine all DataFrames into one
data = pd.concat(li, axis=0, ignore_index=True)

In [62]:
# Group by year and nationality, then count occurrences
nationality_trends = data.groupby(['year', 'Nationality or Origin']).size().unstack(fill_value=0)

# Fill any NaN values with 0 (in case some nationalities do not appear in some years)
nationality_trends = nationality_trends.fillna(0)

print(nationality_trends)


Nationality or Origin  Argentina  Australia  Barbados  Belgium  Canada  \
year                                                                     
2000                           0          2         0        0       2   
2001                           0          0         0        0       1   
2002                           0          1         0        0       7   
2003                           0          0         0        0       2   
2004                           0          2         0        0       4   
2005                           0          0         1        1       1   
2006                           0          0         2        0       4   
2007                           0          0         3        0       6   
2008                           0          0         4        0       2   
2009                           0          1         1        0       5   
2010                           0          1         4        0       5   
2011                           0      

For presenting more readable visualization, we choose the most frequently appeared 10 countries to present in our charts. The list of top 10 countiries can be printed, but their corresponding numbers of songs can only be poorly visualized with default printing. So in the next steps, let's use our visualizing tools for more readable result.

In [63]:
# Calculate the total occurrences of each nationality over all years
total_nationalities = data['Nationality or Origin'].value_counts()

# Select the top N nationalities to display
top_n_nationalities = total_nationalities.head(10).index  # Adjust the number as needed

# Filter the trends to include only these top nationalities
filtered_nationality_trends = nationality_trends[top_n_nationalities]

print(top_n_nationalities)
print(filtered_nationality_trends)

Index(['United States of America', 'United Kingdom', 'Canada', 'Barbados',
       'Australia', 'Jamaica', 'France', 'Spain', 'Trinidad and Tobago',
       'Scotland'],
      dtype='object', name='Nationality or Origin')
Nationality or Origin  United States of America  United Kingdom  Canada  \
year                                                                      
2000                                         88               3       2   
2001                                         88               3       1   
2002                                         82               3       7   
2003                                         92               3       2   
2004                                         89               1       4   
2005                                         91               3       1   
2006                                         82               5       4   
2007                                         84               2       6   
2008                          

We first use the `plotly` package to visualize the total numbers of all songs from all listed countries over 24 years with an interactive bar chart. Put your mouse on the bars and you can see the exact numbers.

In [64]:
fig = px.bar(total_nationalities.sort_values(), orientation='h',
             labels={'index': 'Country', 'value': 'Number of Songs'},
             title='Total Number of Songs from Each Country (2000-2023)')
fig.update_layout(
    xaxis_title='Number of Songs',
    yaxis_title='Country',
    width=1200, 
    height=800 
)
fig.show()

Then with the same package, we can visualize the trends of how each the numbers of ranked songs from each country have changed over years. This time, for the same of a clear and readable visualization, we only choose the top 10 countries out of all data, and present them in a dynamic time-series plot.

In [94]:
fig = px.line(filtered_nationality_trends, x= filtered_nationality_trends.index, y= filtered_nationality_trends.columns,
              labels={'value': 'Number of Songs on rank', 'variable': 'Country'},
              title='National Trends in Billboard Hot 100 (2000-2023)')
fig.update_layout(
    width=1000, 
    height=800)
fig.show()

Too many lines and bars from so many countries and things are getting messier? We can reorganize our data with the library `pycountry_convert`, which can automatically convert our countries into the continents where they are located, so that we can continue analyzing our data on the scope of larger regions.
After printin the result, you can see that the 2400 countries where the artists come from are now displayed as their continents.

In [65]:
import pycountry_convert as pc

# Function to get continent code from country name
def country_to_continent(country_name):
    try:
        country_code = pc.country_name_to_country_alpha2(country_name)
        continent_code = pc.country_alpha2_to_continent_code(country_code)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except:
        return 'Other'

data['continent'] = data['Nationality or Origin'].apply(country_to_continent)
print(data['continent'])

0       North America
1              Europe
2       North America
3       North America
4       North America
            ...      
2395    North America
2396    North America
2397           Europe
2398    North America
2399            Other
Name: continent, Length: 2400, dtype: object


I want to transform the information about continents into regions, to be more precise and avoid ambiguity. In furthur data manipulation, we will only talk about the "regions" where the artists and music come from.

In [66]:
continent_to_region = {
    'Africa': 'Africa',
    'Asia': 'Asia',
    'Europe': 'Europe',
    'North America': 'North America',
    'Oceania': 'Oceania',
    'South America': 'South America',
    'Antarctica': 'Other'  # Typically not relevant for music charts
}

# Apply the mapping to the continent data
data['region'] = data['continent'].map(continent_to_region)

# Group by year and region, then count the occurrences
region_trends = data.groupby(['year', 'region']).size().unstack(fill_value=0)
print(data['region'])


0       North America
1              Europe
2       North America
3       North America
4       North America
            ...      
2395    North America
2396    North America
2397           Europe
2398    North America
2399              NaN
Name: region, Length: 2400, dtype: object


Now let's visualize the trends over 24 years with `plotly`.

In [95]:
fig = px.line(region_trends, x=region_trends.index, y=region_trends.columns,
              labels={'value': 'Number of Songs', 'variable': 'Region'},
              title='Regional Trends in Billboard Hot 100 (2000-2023)')
fig.update_layout(
    width=800, 
    height=1000)
fig.show()

# 2.2 Analysis of genders of the artists

Let's prepare our data about gender, and have a quick look at the overall counts and percentages of each gender. The data cell `Unknown` indicates that the artists are bands instead of individuals.

In [76]:
gender_list = []

for file in all_files:
    df = pd.read_csv(file, usecols=['Gender']) 
    year = int(file[-8:-4])  # Extract the year from the filename
    df['year'] = year  # Add a 'year' column based on the filename
    gender_list.append(df)

# Concatenate all DataFrames into one
gender_data = pd.concat(gender_list, ignore_index=True)
gender_data.to_csv('overall_gender_data.csv', index=False)

gender_counts = gender_data['Gender'].value_counts()

# Calculate the percentage of total for each gender category
gender_percentages = (gender_counts / gender_counts.sum()) * 100

print(gender_counts)
print(gender_percentages)

Gender
male           1206
female          648
Unknown         517
genderfluid      16
non-binary       12
trans woman       1
Name: count, dtype: int64
Gender
male           50.250000
female         27.000000
Unknown        21.541667
genderfluid     0.666667
non-binary      0.500000
trans woman     0.041667
Name: count, dtype: float64


before moving further we'd better remove the `Unknown` data cells since currently there aren't much we can do about the gender identify of a band.

In [78]:
# Filter out rows where gender is 'Unknown'
filtered_gender_data = gender_data[gender_data['Gender'] != 'Unknown']
print(filtered_gender_data)

      Gender  year
0       male  2018
1       male  2018
2     female  2018
3     female  2018
4       male  2018
...      ...   ...
2394    male  2015
2395    male  2015
2396    male  2015
2398    male  2015
2399    male  2015

[1883 rows x 2 columns]


Now we only have 1883 rows of data left, and we only talk about the gender data of **artists as individuals**. Let's make the simple counts and percentages calculation again.

In [79]:
filtered_gender_counts = filtered_gender_data['Gender'].value_counts()

# Calculate the percentage of total for each gender category
filtered_gender_percentages = (filtered_gender_counts / filtered_gender_counts.sum()) * 100

print(filtered_gender_counts)
print(filtered_gender_percentages)

Gender
male           1206
female          648
genderfluid      16
non-binary       12
trans woman       1
Name: count, dtype: int64
Gender
male           64.046734
female         34.413170
genderfluid     0.849708
non-binary      0.637281
trans woman     0.053107
Name: count, dtype: float64


Now we can finally visualize the giant gender gap! Firstly with a pie chart presenting the percentages of the overall data.

In [85]:
fig = px.pie(
    names=filtered_gender_counts.index,
    values=filtered_gender_counts.values,
    title='Cumulative Gender Representation in Billboard Hot 100 (2000-2023)',
    labels={'names': 'Gender', 'values': 'Number of Songs'} 
)

fig.update_traces(textinfo='percent+label')

fig.show()

Then we create a stacked bar chart, to visualize the percentages of different genders of individual artists from each year, and see how things change.

In [83]:
import plotly.graph_objects as go

# Group by year and gender, then count occurrences
annual_distribution = filtered_gender_data.groupby(['year', 'Gender']).size().unstack(fill_value=0)

fig = go.Figure()

for gender in annual_distribution.columns:
    fig.add_trace(go.Bar(
        x=annual_distribution.index,
        y=annual_distribution[gender],
        name=gender
    ))

fig.update_layout(
    barmode='stack',
    title='Annual Gender Distribution in Billboard Hot 100',
    xaxis_title='Year',
    yaxis_title='Number of Songs',
    legend_title='Gender',
    xaxis=dict(type='category')  
)

fig.show()


It is also interesting to see that in our filtered data, the remaining gender data of individual artists are generally increasing over 24 years, indicating that the numbers of music bands on the Hot 100 lists (whose genders are filtered because they are `Unknown`) are decreasing.

## 2.3 Who are the most shining pop stars after 2000?

In [88]:
artist_list = []

# Loop through the files and read the necessary data
for file in all_files:
    df = pd.read_csv(file, usecols=['Artist'])
    year = int(file[-8:-4])  # Extract the year from the filename
    df['year'] = year  # Add a 'year' column based on the filename
    artist_list.append(df)

# Concatenate all DataFrames into one
artist_data = pd.concat(artist_list, ignore_index=True)
print(artist_data)

                                         Artist  year
0                                         Drake  2018
1                                    Ed Sheeran  2018
2     Bebe Rexha featuring Florida Georgia Line  2018
3           Camila Cabello featuring Young Thug  2018
4               Post Malone featuring 21 Savage  2018
...                                         ...   ...
2395             Nicky Jam and Enrique Iglesias  2015
2396                    Ne-Yo featuring Juicy J  2015
2397                              One Direction  2015
2398                                      Drake  2015
2399                Calvin Harris and Disciples  2015

[2400 rows x 2 columns]


Whose songs appeared in the Hot 100 rankings most frequently? Let's make a simple counting and figure out.

In [87]:
# Count the occurrences of each artist
artist_counts = artist_data['Artist'].value_counts()
print(artist_counts)

Artist
Taylor Swift                            28
Rihanna                                 22
Drake                                   20
Pink                                    19
Kelly Clarkson                          17
                                        ..
The Killers                              1
Weezer                                   1
Ciara featuring Ludacris                 1
Lil Jon featuring Usher and Ludacris     1
Calvin Harris and Disciples              1
Name: count, Length: 1336, dtype: int64


Which pop stars have the longest dominance on the Hot 100 rankings?

In [93]:
# Group by artist and find the first and last year they appeared
artist_years = artist_data.groupby('Artist')['year'].agg(['min', 'max'])

# Calculate the duration of time each artist has been on the Billboard Hot 100
artist_years['duration'] = artist_years['max'] - artist_years['min']

sorted_artists = artist_years.sort_values(by='duration', ascending=False).head(50)

print(sorted_artists)



                          min   max  duration
Artist                                       
Beyoncé                  2004  2023        19
Pink                     2000  2019        19
Rihanna                  2005  2023        18
Mariah Carey             2005  2023        18
Blake Shelton            2001  2019        18
Chris Brown              2006  2023        17
Kenny Chesney            2001  2018        17
Taylor Swift             2007  2023        16
Maroon 5                 2004  2020        16
Tim McGraw               2000  2016        16
Miley Cyrus              2008  2023        15
OneRepublic              2007  2022        15
Justin Timberlake        2002  2017        15
Jason Aldean             2009  2023        14
Drake                    2009  2023        14
Lady Gaga                2009  2023        14
Justin Bieber            2009  2022        13
Eminem                   2000  2013        13
Coldplay                 2003  2016        13
Kelly Clarkson           2002  201